In [ ]:
import json
import requests
import os

In [ ]:
wakatime_key = os.environ.get('WAKATIME_API_KEY')
beginning_date = '2023-03-07'
end_date = '2023-03-11'
url = f'https://wakatime.com/api/v1/users/current/summaries?start={beginning_date}&end={end_date}&api_key={wakatime_key}'
url

In [ ]:
response = requests.get(url)
data = response.json()
print(data['cumulative_total'])
print(data['daily_average'])

In [ ]:
print(data['cumulative_total']['text'])
print(data['daily_average']['text'])

In [ ]:
print(data['cumulative_total']['seconds'])
print(data['daily_average']['seconds'])

# Leader Api

In [ ]:
leader_url = f'https://wakatime.com/api/v1/leaders?country_code=US&api_key={wakatime_key}'
leader_url

In [ ]:
leader_response = requests.get(leader_url)
leader_data = leader_response.json()
leaderboard_rank = leader_data['current_user']['rank']
print(leaderboard_rank)

# All Time Count API

In [ ]:
all_time_url = f'https://wakatime.com/api/v1/users/current/all_time_since_today?country_code=US&api_key={wakatime_key}'
all_time_url

In [ ]:
all_time_response = requests.get(all_time_url)
all_time_data = all_time_response.json()
all_time_seconds = all_time_data['data']['total_seconds']
print(all_time_seconds)
all_time_text = all_time_data['data']['text']
print(all_time_text)

# Google Sheets

In [ ]:
import json
import requests
import os
from app.app import append_values_to_google_sheet, wakatime_api_all_time_count, batch_update_google_sheets


WAKATIME_ALL_TIME_SHEET_ID = os.environ.get("WAKATIME_ALL_TIME_SHEET_ID")
SPREADSHEET_ID = os.environ.get("SPREADSHEET_ID")

WAKATIME_ALL_TIME_SHEET_ID, SPREADSHEET_ID

In [ ]:
all_time_response = append_values_to_google_sheet(
    SPREADSHEET_ID,
    WAKATIME_ALL_TIME_SHEET_ID,
    "USER_ENTERED",
    [[1,12,'1900-02-02']],
)
print(all_time_response)

In [ ]:
all_time_data = wakatime_api_all_time_count()
all_time_seconds = all_time_data["all_time_seconds"]
all_time_text = all_time_data["all_time_text"]
all_time_seconds, all_time_text

In [ ]:
all_time_response = append_values_to_google_sheet(
    SPREADSHEET_ID,
    WAKATIME_ALL_TIME_SHEET_ID,
    "USER_ENTERED",
    [
        [
            all_time_seconds,
            all_time_text,
            start_date,
        ]
    ],
)

I'm having issues with this function in the cloud, i think it's because google cloud doesn't like super fast subsequent calls to their api. I'm going to try to stuff all of the updates into one batch update command and see if that works better.

In [ ]:
testing_response = batch_update_google_sheets(
    [[1,12,'1900-02-02']],
    SPREADSHEET_ID,
    # WAKATIME_ALL_TIME_SHEET_ID,
)
print(testing_response)

# Manual Build
See if i can put together this manual stuff

In [ ]:
from app.app import (
    batch_update_google_sheets,
    create_append_request_obj,
    create_rowdata,
    create_string_value_obj,
)

In [ ]:
value_obj = create_string_value_obj(123)
print(type(value_obj))
value_obj

In [ ]:
row_values = ['abc1','2','3']
row_value_objs = [create_string_value_obj(val) for val in row_values]
row_value_objs

In [ ]:
row_data = create_rowdata(row_value_objs)
row_data

In [ ]:
append_request_obj = create_append_request_obj(0,row_data)
append_request_obj

In [ ]:
batch_update_google_sheets([append_request_obj, append_request_obj])

# Date Manipulation

In [ ]:
from datetime import datetime, timedelta

In [ ]:
def get_7_day_range(date):
    date_format = '%Y-%m-%d'
    end_date = date - timedelta(days=7)
    return date.strftime(date_format), end_date.strftime(date_format)

In [ ]:
start_date, end_date = get_7_day_range(datetime.now())
print(start_date)
print(end_date)